In [1]:
import io
import subprocess
import os
import glob
import sys
import pwd

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings
import statsmodels.stats.multitest as smm
import json

from pathlib import Path
#from midas import m
#from datetime import datetime, date
#import altair as alt

# gh specific
sys.path.append('/ghsfa/projects/pharma/shared_scripts/')
import bibp.functions as functions
import hgvslib.pHGVS as pHGVS  ## do a pull on the hgvslib git repo

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025"

# SOW172 TFv2 update (different specificity cutoffs 98%, 97%, 95% and 90%)

## Data preocessing

In [2]:
# Readin latest batches (TFv2 updated version by Evan) from Google drive

# Infinity batch
# df_sow172_all = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20241114_AstraZeneca_SOW202_Batch1_80Samples_InfinityReport.csv", header = 1)
df_sow172_all = pd.read_csv(f"{work_dir}/20240620_AstraZeneca_SOW172_Batch1_340Samples_TFv2_InfinityReport.csv", header = 0) 


print("Total sample size of SOW172 Infinity cumulative batches is " + str(df_sow172_all.shape[0]) + "\n") 
print("Total unique sample size of SOW172 Infinity cumulative batches is " + str(df_sow172_all.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of SOW172 Infinity cumulative batches is " + str(df_sow172_all.Patient_ID.nunique()) + "\n")

df_sow172_all.head()

Total sample size of SOW172 Infinity cumulative batches is 11731

Total unique sample size of SOW172 Infinity cumulative batches is 340

Total unique patient sample size of SOW172 Infinity cumulative batches is 281



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,JAK1,1,65348981.0,3.0,I62V,T>C,c.184A>G,NM_002227.4,51.34,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV99071479,rs202021264,not_provided,138287.0,NaN,NaN,1291.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
2,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,EML4,2,42510096.0,8.0,T309P,A>C,c.925A>C,NM_019063.5,51.10,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1722.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
3,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,MRE11,11,94204784.0,8.0,G267G,T>C,c.801A>G,NM_005590.4,50.65,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs769856404,NaN,NaN,NaN,NaN,1448.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
4,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,VEGFB,11,64005105.0,6.0,*208Yext*?,G>C,c.624G>C,NM_003377.5,50.59,NaN,germline,stop_lost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs1033683825,NaN,NaN,NaN,NaN,1188.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable


In [3]:
# Remove duplication
df_sow172_dedup = df_sow172_all.drop_duplicates(subset = "GHSampleID")
df_sow172_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
27,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable
28,N4496000000,17-102402_1,A0931213,B00578814,17-102402,1,1,SUCCESS,NaN,SNV,NaN,PTPRT,20,41420014.0,3.0,I103L,T>G,c.307A>C,NM_133170.4,53.29,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs200831045,NaN,NaN,NaN,NaN,3235.0,2.39,NaN,NaN,Detected,0.056752,12.15,Low,Not Detected,20.02,3.5,NaN,12/14/23,4/4/23,3/15/24,LUNG,Not Applicable,Not Applicable
67,N4496000000,1044-4_0,A0931214,B00578812,1044-4,0,1,SUCCESS,NaN,SNV,NaN,PIM1,6,37138127.0,1.0,P17S,C>T,c.49C>T,NM_001243186.1,53.69,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs962945709,NaN,NaN,NaN,NaN,599.0,11.50,NaN,NaN,Not Detected,0.000000,6.62,Low,Not Detected,16.59,4.0,NaN,12/19/23,NaN,3/15/24,LUNG,Not Applicable,Not Applicable
106,N4496000000,24002_1,A0931215,B00578817,24002,1,1,SUCCESS,NaN,SNV,NaN,FAT1,4,187540610.0,10.0,T2377M,G>A,c.7130C>T,NM_005245.4,54.12,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV99081608,rs201363601,NaN,NaN,NaN,NaN,2197.0,2.24,NaN,NaN,Detected,0.060444,NaN,Not evaluable,Not Detected,18.30,4.0,NaN,12/14/23,1/17/23,3/15/24,LUNG,Not Applicable,Not Applicable


In [4]:
# Pull "runid" from ghdb
df_sow172_1 = df_sow172_dedup.copy()

# Define the IDs you want to query for
ids = tuple(df_sow172_1['GHSampleID'].unique())

# Generate the SQL query string
query = "SELECT * FROM gh_sample WHERE run_sample_id IN {}".format(ids)

# Get the dataframe with "runid"
sample_df = functions.run_sql_query(query)
sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# Extract the date from runid and convert to datetime format for accurate comparison
sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
df_sow172_1 = df_sow172_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
df_sow172_1 = df_sow172_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
df_sow172_1 = df_sow172_1.drop(columns='run_date')

print("Total sample size of SOW172 Infinity cumulative batches after merging runid is " + str(df_sow172_1.shape[0])) 
df_sow172_1.head()


Total sample size of SOW172 Infinity cumulative batches after merging runid is 340


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7


In [5]:
def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        if folders:
            for folder in folders:
                primary_fc_dir = folder
                # Check if there is a 'sirius' folder under the primary fc_dir. This is for Sirius 1.5 folder structure change.
                if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                    primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
                df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_sow172_2, df_no_found = add_fc_dir_column(df_sow172_1)

print("Total sample size of SOW172 Infinity cumulative batches with fc_dir is " + str(df_sow172_2.shape[0]) + "\n") 
print("Total sample size of SOW172 Infinity cumulative batches with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_sow172_2.head()



Total sample size of SOW172 Infinity cumulative batches with fc_dir is 340

Total sample size of SOW172 Infinity cumulative batches with NO fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7


In [8]:
# CRC caller run test by using Sirius 1.6.x way (i.e. by BIP-epigenomic module specifying the yaml file for input regions)
df_sow172_test = df_sow172_2.head(20)

## Generate S3 MB caller outputs for this SOW using Sirius 1.6.x yaml (CRC ONLY)
###### Git clone Sirius-1.6.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.6.0_Release_Branch/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L653-L680
###### Confluence page to follow: https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update
###### This is from Mingyang's notes: https://guardanthealth.atlassian.net/wiki/spaces/PHARMA/pages/3769729300/Sirius-1.5+vs+Sirius-1.6+and+module+rerun#How-to-run-the-MB-and-TFv2_gamma-callers:~:text=How%20to%20run%20the%20MB%20and%20TFv2_gamma%20callers

In [10]:
# Instead of using epicaller.methylation.methyl_caller funciton, bip-epigenomics module (branch 2.5 with Sirius 1.6 MB caller region/control/module files) is prefered.

def run_bip_epigenomics_module_wrapper_slurm(sample_meta_df, workdir):
    module_dir = '/ghsfa/projects/omni_v2/users/chlee/bip4/bip-epigenomics/' # branch 2.5
    # parameter_set_yaml = "/ghsfa/projects/pharma/customers/OHSU/Infinity/OHSU01_IST01_Infinity/OHSU01_IST01_Infinity_01/data/update_MB_TF/bip-products/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml" # branch Sirius 1.6
    parameter_set_yaml_with_updated_crc_region_file = "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/Sirius_LDT_Soft_absolute.yaml" # branch Sirius 1.6.x file but only works for CRC as rest of the relative path won't work.

    jobs_dir = f"{workdir}/MB_module_CRC_test_jobs"
    output_dir = f"{workdir}/MB_module_CRC_test_output"

    # Check and create directories if not exists
    if not os.path.exists(jobs_dir):
        os.makedirs(jobs_dir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        fc_dir = row["fc_dir"]

        # # Check if there is a 'sirius' folder under the fc_dir. This is for Sirius 1.5 update
        # if os.path.exists(os.path.join(fc_dir, "sirius")):
        #     fc_dir = os.path.join(fc_dir, "sirius")

        mbd_hyper_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_hyper_molecules.tsv"
        bip_config_json_file = f"{fc_dir}/bip_config.json"
        
        if not os.path.exists(mbd_hyper_molecules):
            continue

        # Make a folder for each run_sample_id
        output_sample_dir = f"{output_dir}/{run_sample_id}"

        if not os.path.exists(output_sample_dir):
            os.makedirs(output_sample_dir)

        # Copy the bip_config.json for each sample from BIP output. This is essential for module running
        # (samples in the same batch shared the same bip_config.json)
        os.system(f"cp {bip_config_json_file} {output_sample_dir}")

        job_file = f"{jobs_dir}/MB_call_{run_sample_id}.job"
        shebang_content = (f"#!/bin/bash\n"
                           f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
                           f"#SBATCH --nodes=1\n"
                           f"#SBATCH --ntasks=1\n"
                           f"#SBATCH --cpus-per-task=1\n"
                           f"#SBATCH --mem=60gb\n"
                           f"#SBATCH --partition=dev\n"
                           f"#SBATCH --time=01:00:00\n"
                           f"#SBATCH --output={jobs_dir}/MB_call_{run_sample_id}.log\n\n")
        
        with open(job_file, "w") as job_output:
            job_output.write(shebang_content)
            job_output.write(f"{module_dir}/module_entry msre_caller " + 
                             f"--in-hyper-molecules {mbd_hyper_molecules} " + 
                             f"--sample-id {run_sample_id} " + 
                             f"--output-dir {output_sample_dir}/ " +
                             f"--parameter-set {parameter_set_yaml_with_updated_crc_region_file} " + 
                             f"--threads 16 " + 
                             f"--memory-log {output_sample_dir}/{run_sample_id}-epigenomics_msre_caller.memory.log " +
                             f"--log-file {output_sample_dir}/{run_sample_id}--epigenomics_msre_caller.log")
            # print(job_output)    
        os.system(f"sbatch {job_file}")


In [11]:
#run the function to generate all the MB S3 caller ouputs for this sow  
run_bip_epigenomics_module_wrapper_slurm(df_sow172_test, 
                      f"{work_dir}")

Submitted batch job 97795075
Submitted batch job 97795076
Submitted batch job 97795077
Submitted batch job 97795078
Submitted batch job 97795079
Submitted batch job 97795080
Submitted batch job 97795081
Submitted batch job 97795082
Submitted batch job 97795083
Submitted batch job 97795084
Submitted batch job 97795085
Submitted batch job 97795086
Submitted batch job 97795087
Submitted batch job 97795088
Submitted batch job 97795089
Submitted batch job 97795090
Submitted batch job 97795091
Submitted batch job 97795092
Submitted batch job 97795093
Submitted batch job 97795094


In [16]:
# Retrieve MB S3 CRC caller score and MB S3 Lung caller score based on the cancer type

df_sow172_3 = df_sow172_test.copy()

df_sow172_3["MB_CRC_call_sirius150_region_file"] = np.nan
df_sow172_3["MB_CRC_score_sirius150_region_file"] = np.nan

mb_caller_result_folder = f"{work_dir}/MB_module_CRC_test_output"
problematic_rows = []  # List to store indices of problematic rows

for index, crc_row in df_sow172_3.iterrows():
    run_sample_id = crc_row["GHSampleID"]
    msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}/{run_sample_id}/{run_sample_id}.msre_crc_call.hdr.tsv"   
    try:
        msre_df = pd.read_table(msre_mr_file)

        if not msre_df.empty:
            df_sow172_3.at[index, "MB_CRC_score_sirius150_region_file"] = msre_df.loc[0, "methyl_score"]
            df_sow172_3.at[index, "MB_CRC_call_sirius150_region_file"] = "Detected" if msre_df.loc[0, "methyl_call"] == 1 else "Not Detected"
        else:
            # Add the index to the list if msre_df is empty
            problematic_rows.append(index) 
            
    except Exception as e:
        # Add the index to the list if there is any other exception
        problematic_rows.append(index)  
        print(f"Error processing file {msre_mr_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_sow172_3.loc[problematic_rows]

print("Total sample size of SOW172 Infinity cumulative batches with NO MB output is " + str(df_problematic.shape[0]) + "\n") 

# Returning the DataFrame with problematic rows
df_sow172_3.head()


Total sample size of SOW172 Infinity cumulative batches with NO MB output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_CRC_call_sirius150_region_file,MB_CRC_score_sirius150_region_file
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,Detected,9.494824
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,Detected,-16.814248
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,Not Detected,-21.232218
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,Not Detected,-20.541978
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,Detected,15.589967


## Generate S3 MB CRC caller outputs using epicaller way (Sirius 1.5.0)
###### Git clone Sirius-1.5.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.5.0-RLS/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L610-L623
###### Cloned Sirius-1.5.0_Release_Branch dir: /home/hazhang/projects/Repo/bip-products/

In [6]:
# Make sure update the epicaller to 2.0.3 or above version
import epicaller
print(epicaller.__version__)

2.0.3


In [7]:
import epicaller
import epicaller.methylation.methyl_caller

def mb_crc_caller_sirius150_wrapper_slurm(sample_meta_df, workdir):
    region_file='/home/hazhang/projects/Repo/bip-products/models/Sirius/mafband_crc.classification_space.tsv'
    control_region_file='/home/hazhang/projects/Repo/bip-products/models/Sirius/mafband_crc.positive_control_regions.tsv'
    mr_model_file='/home/hazhang/projects/Repo/bip-products/models/Sirius/mafband_crc.threshold_set_to_zero.tsv'

    jobs_dir = f"{workdir}/MB_CRC_log"
    output_dir = f"{workdir}/MB_CRC_output"

    # Check and create directories if not exists
    if not os.path.exists(jobs_dir):
        os.makedirs(jobs_dir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        fc_dir = row["fc_dir"]
        flowcell_id = row["runid"]
        mbd_hyper_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_hyper_molecules.tsv"
        
        # Check if the molecule file exists
        if not os.path.exists(mbd_hyper_molecules):
            continue

        job_file = f"{jobs_dir}/MB_call_{run_sample_id}.job"
        shebang_content = (f"#!/bin/bash\n"
                           f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
                           f"#SBATCH --nodes=1\n"
                           f"#SBATCH --ntasks=1\n"
                           f"#SBATCH --cpus-per-task=1\n"
                           f"#SBATCH --mem=30gb\n"
                           f"#SBATCH --partition=dev\n"
                           f"#SBATCH --time=00:10:00\n"
                           f"#SBATCH --output={jobs_dir}/MB_call_{run_sample_id}.log\n\n")

        with open(job_file, "w") as job_output:
            job_output.write(shebang_content)
            job_output.write("python3 -m epicaller.methylation.methyl_caller " + 
                             f"--hyper-molecule-file {mbd_hyper_molecules} " + 
                             f"--region-file {region_file} " +
                             f"--control-region-file {control_region_file} " + 
                             f"--model-file {mr_model_file} " +
                             "--model-name crc --min-frag-len 120 --max-frag-len 240 --min-cg-count 12 --max-cg-count 30 --cutsite-count-cutoff 1 --min-mapq 60 " + 
                             f"--sample-id {run_sample_id} --flowcell-id {flowcell_id} --outdir {output_dir}/")
            # print(job_output)    
        os.system(f"sbatch {job_file}")

In [8]:
# Generate the MB S3 CRC caller outputs (Sirius 1.5.0)
mb_crc_caller_sirius150_wrapper_slurm(df_sow172_2, f"{work_dir}")

Submitted batch job 97780752
Submitted batch job 97780753
Submitted batch job 97780754
Submitted batch job 97780755
Submitted batch job 97780756
Submitted batch job 97780757
Submitted batch job 97780758
Submitted batch job 97780759
Submitted batch job 97780760
Submitted batch job 97780761
Submitted batch job 97780762
Submitted batch job 97780763
Submitted batch job 97780764
Submitted batch job 97780765
Submitted batch job 97780766
Submitted batch job 97780767
Submitted batch job 97780768
Submitted batch job 97780769
Submitted batch job 97780770
Submitted batch job 97780771
Submitted batch job 97780772
Submitted batch job 97780773
Submitted batch job 97780774
Submitted batch job 97780775
Submitted batch job 97780776
Submitted batch job 97780777
Submitted batch job 97780778
Submitted batch job 97780779
Submitted batch job 97780780
Submitted batch job 97780781
Submitted batch job 97780782
Submitted batch job 97780783
Submitted batch job 97780784
Submitted batch job 97780785
Submitted batc

In [46]:
# Retrieve MB S3 CRC caller score and MB S3 Lung caller score based on the cancer type
df_sow172_3 = df_sow172_2.copy()

df_sow172_3["MB_score"] = np.nan

mb_caller_result_folder = f"{work_dir}/MB_module_SOW172_output"
crc_caller_result_folder = f"{work_dir}/MB_CRC_output"

problematic_rows = []  # List to store indices of problematic rows

for index, crc_row in df_sow172_3.iterrows():
    run_sample_id = crc_row["GHSampleID"]

    # This is a mixed cancer type cohort
    cancer_type = crc_row["Cancertype"]
    if cancer_type == "CRC":
        msre_mr_file = f"{crc_caller_result_folder}/{run_sample_id}.msre_crc_call.hdr.tsv"
    else:
        msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}/{run_sample_id}/{run_sample_id}.msre_lung_call.hdr.tsv"
    try:
        msre_df = pd.read_table(msre_mr_file)

        if not msre_df.empty:
            df_sow172_3.at[index, "MB_score"] = msre_df.loc[0, "methyl_score"] 

        else:
            # Add the index to the list if msre_df is empty
            problematic_rows.append(index) 
            
    except Exception as e:
        # Add the index to the list if there is any other exception
        problematic_rows.append(index)  
        print(f"Error processing file {msre_mr_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_sow172_3.loc[problematic_rows]

print("Total sample size of SOW172 Infinity cumulative batches with NO MB output is " + str(df_problematic.shape[0]) + "\n") 

# Returning the DataFrame with problematic rows
df_sow172_3.head()


Total sample size of SOW172 Infinity cumulative batches with NO MB output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_score
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,29.694824
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,3.385752
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-21.253684
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-0.341978
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,35.789967


In [49]:
# To convert the mb crc score to original value. Cutoff was -20.2
df_sow172_crc = df_sow172_3.copy()

df_sow172_crc['MB_score'] = np.where(
    df_sow172_crc['Cancertype'] == 'CRC',  
    df_sow172_crc['MB_score'] - 20.2,  
    df_sow172_crc['MB_score'] 
)

df_sow172_crc.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_score
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,9.494824
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,-16.814248
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-21.253684
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-20.541978
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,15.589967
